In [1]:
import pandas as pd
from datetime import datetime
import re
from pytz import timezone
import psycopg2


In [ ]:
try:
    # Connect to database
    connection = psycopg2.connect(
        user="user",
        port="port",
        host="localhost",
        database="name"
    )
    
    cursor = connection.cursor()
    
    queries = [
        "SELECT * FROM sentiment_data_aapl;",
        "SELECT * FROM sentiment_data_amzn;",
        "SELECT * FROM sentiment_data_msft;",
        "SELECT * FROM sentiment_data_nvidia;"
    ]
    
    aapl_data, amzn_data, msft_data, nvidia_data = None, None, None, None
    
    # Execute the queries and fetch data
    cursor.execute(queries[0])
    aapl_data = cursor.fetchall()

    cursor.execute(queries[1])
    amzn_data = cursor.fetchall()
    
    cursor.execute(queries[2])
    msft_data = cursor.fetchall()

    cursor.execute(queries[3])
    nvidia_data = cursor.fetchall()
    
except Exception as error:
    print(f"Failed to fetch records: {error}")

finally:
    # Close the connection
    if connection:
        cursor.close()
        connection.close()
        print("Connection closed.")


In [28]:
nvidia_data["Time_Published"]

0                       January 8, 2009 Thursday
1         January 28, 2009 Wednesday 2:36 PM EST
2            January 2, 2009 Friday 12:15 PM EST
3            January 12, 2009 Monday 9:01 AM EST
4         January 14, 2009 Wednesday 4:17 PM EST
                          ...                   
33430    January 12, 2011 Wednesday 09:16 AM EST
33431        January 14, 2011 Friday 9:32 AM GMT
33432                     January 7, 2011 Friday
33433      January 5, 2011 Wednesday 7:00 PM GMT
33434     January 12, 2011 Wednesday 8:34 AM GMT
Name: Time_Published, Length: 33435, dtype: object

In [45]:
# Function to remove days of the week, AM/PM, and timezones from date string
def remove_extras(date_str):
    if pd.isna(date_str) or date_str is None:
        return None
    cleaned_date = re.sub(r'(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday|EST|EDT|PST|PDT|GMT|AM|PM)', '', date_str).strip()
    return cleaned_date

# Function to convert cleaned date string to datetime object
def to_datetime(date_str):
    if pd.isna(date_str) or date_str is None:
        return None
    try:
        return pd.to_datetime(date_str, errors='coerce')
    except Exception as e:
        print(f"Could not convert to datetime: {date_str}, error: {e}")
        return None

# Remove extras and convert to datetime for each dataframe
for df in [aapl_data, amzn_data, msft_data, nvidia_data]:
    df['Cleaned_Date'] = df['Time_Published'].apply(remove_extras)
    df['Datetime'] = df['Cleaned_Date'].apply(to_datetime)



/Users/rithvikprakki/anaconda3/lib/python3.10/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname CET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


KeyboardInterrupt: 

In [56]:
def convert_to_est_datetime(date_str):
    if pd.isna(date_str):
        return None
    
    # Remove day names
    date_str = re.sub(r'(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday)', '', date_str).strip()
    
    # Timezone conversion map
    tz_conversion = {
        'EST': timezone('US/Eastern'),
        'EDT': timezone('US/Eastern'),
        'PST': timezone('US/Pacific'),
        'PDT': timezone('US/Pacific'),
        'GMT': timezone('GMT'),
        'CET': timezone('CET'),
        'MST': timezone('US/Mountain'),
        'EET': timezone('EET'),
        'JST': timezone('Asia/Tokyo')
    }
    
    tz = 'EST'  # Default to EST
    for tz_key in tz_conversion.keys():
        if tz_key in date_str:
            tz = tz_key
            date_str = date_str.replace(tz_key, '').strip()
            break
    
    # Remove redundant AM/PM indicators
    date_str = date_str.replace('AM', ' AM').replace('PM', ' PM').strip()
    
    # Handle 24:xx hour format
    if "24:" in date_str:
        date_str = date_str.replace(" 24:", " 00:").strip()
        add_day = True
    else:
        add_day = False
    
    # Handle different datetime formats
    try:
        naive_dt = datetime.strptime(date_str, "%B %d, %Y %H:%M %p")
    except:
        try:
            naive_dt = datetime.strptime(date_str, "%b %d, %Y")
        except:
            try:
                naive_dt = datetime.strptime(date_str, "%B %d, %Y")
            except Exception as e:
                print(f"Could not convert to datetime: {date_str}, error: {e}")
                return None
    
    if add_day:
        naive_dt += pd.Timedelta(days=1)
    
    # Convert to the specified timezone
    localized_dt = tz_conversion[tz].localize(naive_dt)
    est_dt = localized_dt.astimezone(timezone('US/Eastern'))
    
    return est_dt

# Apply the function
for df in [aapl_data, amzn_data, msft_data, nvidia_data]:
    df['Datetime'] = df['Time_Published'].apply(convert_to_est_datetime)


Could not convert to datetime: July 2, 2007 Issue, error: unconverted data remains:  Issue
Could not convert to datetime: June 2, 2008 Issue, error: unconverted data remains:  Issue


In [61]:
aapl_data = aapl_data.drop("Cleaned_Date", axis = 1)

In [77]:
nvidia_data = nvidia_data.sort_values(by='Datetime', ascending=True)
aapl_data = aapl_data.sort_values(by='Datetime', ascending=True)
amzn_data = amzn_data.sort_values(by='Datetime', ascending=True)
amzn_data = amzn_data.sort_values(by='Datetime', ascending=True)


In [76]:
nvidia_data

,ID,Title,Sentiment_Score,Time_Published,Datetime
343,343,Ponen en una laptop grabador de Blu-ray; Lanza...,1.000000,"January 1, 2007 Monday",2007-01-01 00:00:00-05:00
341,341,Ponen en una laptop grabador de Blu-ray; Hogar...,1.000000,"January 1, 2007 Monday",2007-01-01 00:00:00-05:00
185,185,LOOKING FORWARD: TECHNOLOGY TRENDS IN 2007,1.071429,"January 1, 2007 Monday",2007-01-01 00:00:00-05:00
142,142,Graphics chip co Lucid raises $12m; Intel Capi...,1.066667,"January 1, 2007 Monday",2007-01-01 00:00:00-05:00
166,166,Briefing.com: Tech Stocks,1.297297,"January 1, 2007 Monday 11:55 AM EST",2007-01-01 11:55:00-05:00
...,...,...,...,...,...
33327,33329,"AMD, Nvidia Saw Market Share Rise in Q4 2010: ...",1.136364,"January 31, 2011 Monday 01:04 PM GMT",2011-01-30 20:04:00-05:00
33297,33299,NVIDIA (NVDA) Showing Bullish Technicals With ...,0.923077,"January 31, 2011 Monday 8:53 AM EST",2011-01-31 08:53:00-05:00
33402,33404,The Best Assistant to the Coming Motorola Xoom...,1.117647,"January 31, 2011 Monday 9:08 AM EST",2011-01-31 09:08:00-05:00
2735,2735,No Headline In Original,1.057143,"July 2, 2007 Issue",NaT


In [142]:
def average_daily_sentiment(df):
    df['Date'] = df['Datetime'].dt.date
    
    # Group by Date and average the Sentiment_Score
    df_grouped = df.groupby('Date').agg({'Sentiment_Score': 'mean'}).reset_index()

    df_grouped["Date"] = pd.to_datetime(df_grouped["Date"])
    
    return df_grouped

aapl_data_grouped = average_daily_sentiment(aapl_data)
amzn_data_grouped = average_daily_sentiment(amzn_data)
msft_data_grouped = average_daily_sentiment(msft_data)
nvidia_data_grouped = average_daily_sentiment(nvidia_data)

In [96]:
aapl_data_grouped

,Date,Sentiment_Score
0,2007-01-01,1.061519
1,2007-01-02,1.118282
2,2007-01-03,1.051647
3,2007-01-04,1.020751
4,2007-01-05,0.879535
...,...,...
1167,2010-03-26,1.132178
1168,2010-03-28,1.060930
1169,2010-03-29,1.160856
1170,2010-03-30,0.979191


In [155]:
def filter_data(df):
    return df[(df['Date'] >= '2007-01-01') & (df['Date'] <= '2009-01-31')]

aapl_data_filtered = filter_data(aapl_data_grouped)
amzn_data_filtered = filter_data(amzn_data_grouped)
msft_data_filtered = filter_data(msft_data_grouped)
nvidia_data_filtered = filter_data(nvidia_data_grouped)

In [164]:
import yfinance as yf

tickers = ["AAPL", "AMZN", "NVDA", "MSFT"]
start_date = "2007-01-01"
end_date = "2009-01-31"

stock_data = {}

# Fetch the data
for ticker in tickers:
    stock_data[ticker] = yf.download(ticker, start=start_date, end=end_date)

print(stock_data["AAPL"])


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                Open      High       Low     Close  Adj Close      Volume
Date                                                                     
2007-01-03  3.081786  3.092143  2.925000  2.992857   2.540327  1238319600
2007-01-04  3.001786  3.069643  2.993571  3.059286   2.596712   847260400
2007-01-05  3.063214  3.078571  3.014286  3.037500   2.578219   834741600
2007-01-08  3.070000  3.090357  3.045714  3.052500   2.590951   797106800
2007-01-09  3.087500  3.320714  3.041071  3.306071   2.806182  3349298400
...              ...       ...       ...       ...        ...         ...
2009-01-26  3.173571  3.248929  3.153571  3.201429   2.717361   692238400
2009-01-27  3.221071  3.269643  3.205000  3.240357   2.750

In [170]:
common_dates = set(stock_data["AAPL"].index).intersection(
    set(stock_data["AMZN"].index),
    set(stock_data["MSFT"].index),
    set(stock_data["NVDA"].index)
)

In [171]:
aapl_data_filtered = aapl_data_filtered[aapl_data_filtered['Date'].isin(common_dates)]
amzn_data_filtered = amzn_data_filtered[amzn_data_filtered['Date'].isin(common_dates)]
msft_data_filtered = msft_data_filtered[msft_data_filtered['Date'].isin(common_dates)]
nvidia_data_filtered = nvidia_data_filtered[nvidia_data_filtered['Date'].isin(common_dates)]


In [200]:
aapl_data_filtered['Price'] = aapl_data_filtered['Date'].map(stock_data["AAPL"]["Close"])
amzn_data_filtered['Price'] = amzn_data_filtered['Date'].map(stock_data["AMZN"]["Close"])
msft_data_filtered['Price'] = msft_data_filtered['Date'].map(stock_data["MSFT"]["Close"])
nvidia_data_filtered['Price'] = nvidia_data_filtered['Date'].map(stock_data["NVDA"]["Close"])


/var/folders/6z/_tfc4bdx6w7bg0w4vm5g0j5w0000gn/T/ipykernel_87473/2071334679.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aapl_data_filtered['Price'] = aapl_data_filtered['Date'].map(stock_data["AAPL"]["Close"])


In [212]:
merged_df = amzn_data_filtered.merge(aapl_data_filtered, on='Date', suffixes=('_AMZN', '_AAPL'))\
                     .merge(msft_data_filtered, on='Date', suffixes=('', '_MSFT'))\
                     .merge(nvidia_data_filtered, on='Date', suffixes=('_MSFT', '_NVIDIA'))

In [279]:
merged_df.to_csv("sentiment_and_prices.csv", index = False)